In [1]:
# import modules
import os
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from natsort import natsorted

In [2]:
# read target pairs to annotate
os.chdir('/Users/narod/Documents/GitHub/context_specific_SL_prediction/1_feature_calculation/annotations/output_files')

target_pairs = pd.read_csv('ito_pairs_ranked_ess.csv')
#target_pairs = pd.read_csv('./klingbeil_pairs/klingbeil_pairs_ranked_ess.csv')
#target_pairs = pd.read_csv('./parrish_pairs/parrish_pairs_ranked_ess.csv')
#target_pairs = pd.read_csv('./36K_dataset/36Kdataset_ranked_ess.csv')

In [3]:
target_pairs.head()

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,smallest_gene_expression,smallest_GO_CC_ranked_ess,A1_rank,zA1_rank,A2_rank,zA2_rank,max_ranked_A1A2,min_ranked_A1A2,z_max_ranked_A1A2,z_min_ranked_A1A2
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,NaN,NaN,5108.0,-0.759628,NaN,NaN,5108.0,5108.0,-0.759628,-0.759628
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,NaN,NaN,9125.0,0.303431,NaN,NaN,9125.0,9125.0,0.303431,0.303431
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,NaN,NaN,4063.0,-1.036177,NaN,NaN,4063.0,4063.0,-1.036177,-1.036177
3,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000681,A549_LUNG,0.977988,0.379455,False,...,NaN,NaN,15434.0,1.973047,NaN,NaN,15434.0,15434.0,1.973047,1.973047
4,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000756,GI1_CENTRAL_NERVOUS_SYSTEM,0.999586,-0.077118,False,...,NaN,NaN,6507.0,-0.389397,NaN,NaN,6507.0,6507.0,-0.389397,-0.389397


In [4]:
target_pairs.loc[target_pairs['genepair'] == 'ADAM8_ADAM12']

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,smallest_gene_expression,smallest_GO_CC_ranked_ess,A1_rank,zA1_rank,A2_rank,zA2_rank,max_ranked_A1A2,min_ranked_A1A2,z_max_ranked_A1A2,z_min_ranked_A1A2
1650,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000022,PATU8988S_PANCREAS,0.790599,-0.114212,False,...,NaN,NaN,17328.0,0.734933,4979.0,-1.440645,17328.0,4979.0,0.734933,-1.440645
1651,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000307,PK1_PANCREAS,0.994898,-0.104993,False,...,NaN,NaN,16886.0,0.562421,12710.0,0.888743,16886.0,12710.0,0.888743,0.562421
1652,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000632,HS944T_SKIN,0.834862,-0.170243,False,...,NaN,NaN,16984.0,0.600670,11992.0,0.672406,16984.0,11992.0,0.672406,0.600670
1653,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000681,A549_LUNG,0.445588,-0.513740,False,...,NaN,NaN,13225.0,-0.866465,11612.0,0.557910,13225.0,11612.0,0.557910,-0.866465
1654,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000756,GI1_CENTRAL_NERVOUS_SYSTEM,0.999586,0.049023,False,...,NaN,NaN,17204.0,0.686536,7200.0,-0.771447,17204.0,7200.0,0.686536,-0.771447
1655,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000801,HS936T_SKIN,0.446371,-0.401475,False,...,NaN,NaN,15014.0,-0.168220,13055.0,0.992693,15014.0,13055.0,0.992693,-0.168220
1656,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000881,MELJUSO_SKIN,0.831609,-0.043630,False,...,NaN,NaN,13349.0,-0.818068,6626.0,-0.944396,13349.0,6626.0,-0.818068,-0.944396
1657,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000915,IPC298_SKIN,1.000000,0.010868,False,...,NaN,NaN,17285.0,0.718150,5370.0,-1.322835,17285.0,5370.0,0.718150,-1.322835
1658,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-000987,MEWO_SKIN,0.995681,-0.063444,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659,ADAM8_ADAM12,ADAM8,ADAM12,101,8038,ACH-001524,HSC5_SKIN,0.561370,-0.209467,False,...,NaN,NaN,17407.0,0.765767,11088.0,0.400027,17407.0,11088.0,0.765767,0.400027


In [5]:
# read prediction score data
bp = pd.read_csv('/Users/narod/Documents/GitHub/context_specific_SL_prediction/1_feature_calculation/annotations/input_files/processed_DeKegel_TableS8.csv')
display(bp.loc[bp['genepair'] == 'ADAMTS6_ADAMTS10'])
display(bp.loc[bp['genepair'] == 'ADAM8_ADAM12'])
display(bp.loc[bp['genepair'] == 'USP4_USP15'])

prediction_score = bp[['genepair', 'prediction_score']]

print(f'# of unique genepairs: {prediction_score.genepair.nunique()}')
display(prediction_score[:3])

,prediction_rank,prediction_percentile,old_genepair,genepair,A1,A2,A1_entrez,A2_entrez,A1_ensembl,A2_ensembl,...,shared_ppi_mean_essentiality,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,A1_entrez_new,A2_entrez_new,A1_new,A2_new,A1_ensembl_new,A2_ensembl_new
15506,15507,42.4,ADAMTS10_ADAMTS6,ADAMTS6_ADAMTS10,ADAMTS10,ADAMTS6,81794,11174,ENSG00000142303,ENSG00000049192,...,0.0,0.611649,1.630857,18.696327,11174.0,81794,ADAMTS6,ADAMTS10,ENSG00000049192,ENSG00000142303


,prediction_rank,prediction_percentile,old_genepair,genepair,A1,A2,A1_entrez,A2_entrez,A1_ensembl,A2_ensembl,...,shared_ppi_mean_essentiality,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,A1_entrez_new,A2_entrez_new,A1_new,A2_new,A1_ensembl_new,A2_ensembl_new
16781,16782,45.8,ADAM12_ADAM8,ADAM8_ADAM12,ADAM12,ADAM8,8038,101,ENSG00000148848,ENSG00000151651,...,0.0,0.18273,2.448785,21.963953,101.0,8038,ADAM8,ADAM12,ENSG00000151651,ENSG00000148848


,prediction_rank,prediction_percentile,old_genepair,genepair,A1,A2,A1_entrez,A2_entrez,A1_ensembl,A2_ensembl,...,shared_ppi_mean_essentiality,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,A1_entrez_new,A2_entrez_new,A1_new,A2_new,A1_ensembl_new,A2_ensembl_new
1688,1689,4.7,USP15_USP4,USP4_USP15,USP15,USP4,9958,7375,ENSG00000135655,ENSG00000114316,...,0.46926,0.7735,9.646537,26.546565,7375.0,9958,USP4,USP15,ENSG00000114316,ENSG00000135655


# of unique genepairs: 36648


,genepair,prediction_score
0,SMARCA2_SMARCA4,0.430886
1,EXOC6_EXOC6B,0.410447
2,STAG1_STAG2,0.405952


In [6]:
annotated_target_pairs = pd.merge(target_pairs, prediction_score, how='left', on='genepair')
print(f'# of unique genepairs: {annotated_target_pairs.genepair.nunique()}')
print(f'len of dataframe: {annotated_target_pairs.shape[0]}')
print(f'# of NAs: {annotated_target_pairs.prediction_score.isna().sum()}')
display(annotated_target_pairs[:3])

# of unique genepairs: 4523
len of dataframe: 49753
# of NAs: 3355


,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,smallest_GO_CC_ranked_ess,A1_rank,zA1_rank,A2_rank,zA2_rank,max_ranked_A1A2,min_ranked_A1A2,z_max_ranked_A1A2,z_min_ranked_A1A2,prediction_score
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,NaN,5108.0,-0.759628,NaN,NaN,5108.0,5108.0,-0.759628,-0.759628,0.012559
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,NaN,9125.0,0.303431,NaN,NaN,9125.0,9125.0,0.303431,0.303431,0.012559
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,NaN,4063.0,-1.036177,NaN,NaN,4063.0,4063.0,-1.036177,-1.036177,0.012559


In [7]:
na_target_pairs = annotated_target_pairs.loc[annotated_target_pairs['prediction_score'].isna(), 'genepair'].unique()
print(f'# of unique genepairs with no prediction score: {len(na_target_pairs)}')

# of unique genepairs with no prediction score: 305


In [8]:
pd.set_option("display.max_rows", None)
annotated_target_pairs.isna().sum()

genepair                                0
A1                                      0
A2                                      0
A1_entrez                               0
A2_entrez                               0
DepMap_ID                               0
cell_line                               0
Gemini_FDR                            224
raw_LFC                                 0
SL                                      0
org_A1                                  0
org_A2                                  0
A1_copy_number_data                   168
A1_expression_data                    352
A1_gene_effect_data                  6853
A1_zexpression_data                   352
A1_zgene_effect_data                 6853
A2_copy_number_data                   279
A2_expression_data                    517
A2_gene_effect_data                  7243
A2_zexpression_data                   517
A2_zgene_effect_data                 7243
A1_Deleterious                          0
A1_mut                            

### Save as csv file

In [9]:
os.chdir('/Users/narod/Documents/GitHub/context_specific_SL_prediction/1_feature_calculation/annotations/output_files')

annotated_target_pairs.to_csv('ito_pairs_all.csv', index=False)
#annotated_target_pairs.to_csv('./klingbeil_pairs/klingbeil_pairs_all.csv', index=False)
#annotated_target_pairs.to_csv('./parrish_pairs/parrish_pairs_all.csv', index=False)
#annotated_target_pairs.to_csv('./36K_dataset/36Kdataset_all.csv', index=False)